<a href="https://colab.research.google.com/github/rajhack28/PetNexus-A-Comprehensive-Web-Platform-for-Pet-Information-and-Care/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization, TimeDistributed, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Reshape, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image dimensions
img_width, img_height = 224, 224  # Adjusted to match pre-trained models
num_classes = 14  # 14 hand gestures

# Use VGG16 as a feature extractor
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Uncomment below to use ResNet50 instead
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,  # Use pre-trained base model
    Flatten(),   # Flatten to transition to dense layers
    Reshape((1, -1)),  # Reshape for LSTM compatibility

    # LSTM layer for sequence learning
    LSTM(128, return_sequences=False),  # Increased LSTM units

    # Fully connected layers
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Dropout for regularization

    # Output layer for 14 classes
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training and validation data
batch_size = 32
dataset_path = '/content/drive/My Drive/HG14-Hand Gesture'

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data)
)


Found 11469 images belonging to 14 classes.
Found 2865 images belonging to 14 classes.
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 39/359 ━━━━━━━━━━━━━━━━━━━━ 1:54:37 21s/step - accuracy: 0.1293 - loss: 2.9147